# Cluster Pipeline

This Notebook is providing a Cluster pipeline for aggregated data

In [ ]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug, TobaccoUse,LabValue
from fiber.storage import yaml as fiberyaml
import time
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from functools import reduce
from ppca import PPCA
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
import category_encoders as ce
import json
from sklearn import metrics
from sklearn.decomposition import FastICA
from sklearn.metrics import pairwise_distances
from sklearn.metrics import davies_bouldin_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from mpl_toolkits.mplot3d import Axes3D
from sklearn.manifold import TSNE
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram
from sklearn.cluster import AgglomerativeClustering
from pickle import load
from pickle import dump
import pickle
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import preprocessing
import scipy.cluster.hierarchy as shc
import scipy.stats as stats
import researchpy as rp
from keras.models import Model
from keras.layers import Dense, Input
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split  
from keras.layers import Input, Dense 
from keras.models import Model, Sequential 
from keras import regularizers 
import umap
from sklearn.cluster import DBSCAN
import hdbscan
import plotly.express as px

 # Column Transformer

In [ ]:
#columnTransformer out of the scikit-learn libraby
#applying StandardScaler on numeric values and OneHotEncouder on categorical
def apply_columnTransformer(df,df_name,num_scaler_name,cat_scaler_name,experiment_name):
    #load transformed df and column transformer if already available
    name_transformed_df=df_name+'_'+num_scaler_name+'_'+cat_scaler_name
    try:
         
        with open('Cohort/Models/ColumnTransformer/'+experiment_name+'.pkl', 'rb') as f:
            ctransformer = pickle.load(f)
        print('ctransformer loaded')
        load = np.load('Cohort/Models/ColumnTransformer/'+experiment_name+'.npz')
        transformed_df=load['a']
        print('transformed_df loaded')
            
            
        
        #print(transformed_df)
        return transformed_df, ctransformer
    #if a new df was introduced, apply the column transformation
    except:
        #identify categorical and numerical Columns of the dataframe
        categorical_cols = [c for c in df.columns if df[c].dtype in [np.object,np.str] ]
        numerical_cols = [c for c in df.columns if df[c].dtype in [np.float, np.int] ]
        #select the scaler that should be applied
        if num_scaler_name=='StandardScaler':
            num_scaler=StandardScaler()
        if num_scaler_name=='MinMaxScaler':
            num_scaler=preprocessing.MinMaxScaler()    
        if cat_scaler_name=='OneHotEncoder':
            cat_scaler=ce.OneHotEncoder()
        if cat_scaler_name=='BinaryEncoder':
            cat_scaler=ce.BinaryEncoder(drop_invariant = True, handle_missing = 'return_nan')   
        #apply the Transformer
        ctransformer = ColumnTransformer([
        ('num', num_scaler, numerical_cols),
        ('cat', cat_scaler, categorical_cols)])
        #save the ctransformer and the transformed df       
        dump(ctransformer, open('Cohort/Models/ColumnTransformer/'+experiment_name+'.pkl', 'wb')) 
        transformed_df=ctransformer.fit_transform(df)
        print(transformed_df)
        np.savez_compressed('Cohort/Models/ColumnTransformer/'+experiment_name+'.npz',a=transformed_df)
        
            
        return transformed_df, ctransformer
  
    
    

# Dimension Reduction

## PPCA

In [ ]:
def apply_ppca(df,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name): 
    ppca = PPCA()
    ppca.fit(data=transformed_df, d=dimension, verbose=True)
    transformed_train = ppca.transform()
    print(transformed_train)
    dump(ppca, open('Cohort/Models/DimReduction/'+experiment_name+'.pkl', 'wb')) 
    return transformed_train

## TSNE

In [ ]:
def apply_TSNE(df,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name):
    time_start = time.time()
    tsne = TSNE(n_components=2, verbose=1, perplexity=160, n_iter=1000)
    tsne_results = tsne.fit_transform(transformed_df)
    #tsne_results = tsne.fit_transform(df)
    print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))
    plt.scatter(tsne_results[:,0],tsne_results[:,1])
    dump(tsne_results, open('Cohort/Models/DimReduction/'+experiment_name+'.pkl', 'wb'))
    return tsne_results

## FAST ICA 


In [ ]:
def apply_ICA(df,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name):
    transformer = FastICA(n_components=dimension,random_state=0)
    X_transformed = transformer.fit_transform(transformed_df)
    print(X_transformed.shape)
    dump(transformer, open('Cohort/Models/DimReduction/'+experiment_name+'.pkl', 'wb'))
    return X_transformed

## SVD

In [ ]:
#https://machinelearningmastery.com/singular-value-decomposition-for-dimensionality-reduction-in-python/
#good if data is sparse
#n_iter=5 = Default
def apply_svd(df,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name):
    svd = TruncatedSVD(n_components=dimension, n_iter=5, random_state=42)
    X_transformed=svd.fit_transform(transformed_df)
    #print(X_transformed.shape)
    dump(svd, open('Cohort/Models/DimReduction/'+experiment_name+'.pkl', 'wb'))
    return X_transformed

## PCA

In [ ]:
def apply_pca(df,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name):
    pca = PCA(n_components=dimension)
    X_transformed=pca.fit_transform(transformed_df)

    print()
    print(X_transformed)
    dump(pca, open('Cohort/Models/DimReduction/'+experiment_name+'.pkl', 'wb'))
    return X_transformed

## Incremental PCA

In [ ]:
def apply_ipca(df,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name):
    ipca = IncrementalPCA(n_components=dimension, batch_size=30)
    X_transformed=ipca.fit_transform(transformed_df)
    print(X_transformed)
    dump(ipca, open('Cohort/Models/DimReduction/'+experiment_name+'.pkl', 'wb'))
    return X_transformed

## KernelPCA

In [ ]:
def apply_kpca(df,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name):
    kpca = KernelPCA(n_components=dimension, kernel='linear')
    X_transformed=kpca.fit_transform(transformed_df)
    print(X_transformed)
    dump(kpca, open('Cohort/Models/DimReduction/'+experiment_name+'.pkl', 'wb'))
    return X_transformed

## LDA

In [ ]:
def apply_lda(df,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name):
    lda = LatentDirichletAllocation(n_components=dimension,random_state=0)
    transformed_df=abs(transformed_df)
    X_transformed=lda.fit_transform(transformed_df)
    print(X_transformed)
    dump(lda, open('Cohort/Models/DimReduction/'+experiment_name+'.pkl', 'wb'))
    return X_transformed

# UMAP

In [ ]:
def apply_umap(df,transformed_df,dimension,umap_distance,umap_neighbors,df_name,num_scaler_name,cat_scaler_name,experiment_name,save):
    clusterable_embedding = umap.UMAP(
        n_neighbors=umap_neighbors,
        min_dist=umap_distance,
        n_components=dimension,
        random_state=42,
    )
    X_transformed=clusterable_embedding.fit_transform(transformed_df)
    if save==True:
        dump(clusterable_embedding, open('Cohort/Models/DimReduction/'+experiment_name+'.pkl', 'wb'))
    return X_transformed

## Autoencoder

In [ ]:
def apply_AE(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,a_f_decoder,a_f_encoder,n_layer,batchsize,epochs,optimizer,loss_function):
    X=transformed_df
    #Building the Auto-encoder neural network
    # Building the Input Layer 
    input_layer = Input(shape =(X.shape[1], )) 

    # Building the Encoder network 
    encoded = Dense(50, activation =a_f_encoder)(input_layer) 
    encoded = Dense(dimension, activation =a_f_encoder)(encoded) 
    decoded = Dense(50, activation =a_f_decoder)(encoded) 
    # Building the Output Layer 
    output_layer = Dense(X.shape[1], activation =a_f_decoder)(decoded) 
    
    #Train AE
    # Defining the parameters of the Auto-encoder network 
    autoencoder = Model(input_layer, output_layer) 
    autoencoder.compile(optimizer=optimizer, loss=loss_function) 

    # Training the Auto-encoder network 
    history = autoencoder.fit(X, X,  
                    batch_size = batchsize, epochs = epochs,  
                    shuffle = True, validation_split = 0.20)
    #loss for result excel
    print(history.history['val_loss'][(epochs-1)])
    loss=history.history['val_loss'][(epochs-1)]
    #Retaining the encoder part of the Auto-encoder to encode data

    hidden_representation = Sequential() 
    hidden_representation.add(autoencoder.layers[0]) 
    hidden_representation.add(autoencoder.layers[1])  
    hidden_representation.add(autoencoder.layers[2])
    normal_hidden_rep = hidden_representation.predict(X)
   #dump(history, open('Cohort/Models/DimReduction/'+df_name+'_'+num_scaler_name+'_'+cat_scaler_name+'_AE_'+str(dimension)+'_'+a_f_encoder+'_'+a_f_decoder+'_'+str(n_layer)+'_'+str(batchsize)+'_'+str(epochs)+'_'+optimizer+'_'+loss_function+'.pkl', 'wb'))
    return normal_hidden_rep, loss
            

# Cluster Method

## kmeans 

In [ ]:
def apply_kmeans(transformed_sample,ellbow_method,cluster,df_name,num_scaler_name,cat_scaler_name,dim_red_method,experiment_name):
    if ellbow_method==True:
        elbow_method(transformed_sample)
    #scatter_plot(transformed_sample,None) 
    #plt.scatter(transformed_sample[:,0],transformed_sample[:,1])
    kmeans = KMeans(n_clusters=cluster, init='k-means++', max_iter=5000, n_init=10, random_state=0)
    pred_y = kmeans.fit_predict(transformed_sample)
    #plt.scatter(transformed_sample[:,0], transformed_sample[:,1])
    #plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=300, c='red')
    #plt.show()
    #scatter_plot(transformed_sample,kmeans.labels_)
    '''
    from sklearn.metrics.pairwise import pairwise_distances_argmin
    fig = plt.figure(figsize=(15, 5))
    fig.subplots_adjust(left=0.02, right=0.98, bottom=0.05, top=0.9)
    colors = ['#4EACC5', '#FF9C34', '#4E9A06','#FF0000','#8800FF']
    k_means_labels = pairwise_distances_argmin(transformed_sample, kmeans.cluster_centers_)
    ax = fig.add_subplot(1, 3, 1)
    for k, col in zip(range(cluster), colors):
        my_members = k_means_labels == k
        cluster_center = kmeans.cluster_centers_[k]
        ax.plot(transformed_sample[my_members, 0], transformed_sample[my_members, 1], 'w',
                markerfacecolor=col, marker='.')
        ax.plot(cluster_center[0], cluster_center[1], 'o', markerfacecolor=col,
                markeredgecolor='k', markersize=6)
    experiment_name=experiment_name
    ax.set_title(experiment_name)
    ax.set_xticks(())
    ax.set_yticks(())
    fig.savefig('Cohort/Models/Plots/'+experiment_name+'.png')'''
    return kmeans.labels_

In [ ]:
def elbow_method(transformed_sample): 
    wcss = []
    for i in range(1, 11):
        kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
        kmeans.fit(transformed_sample)
        wcss.append(kmeans.inertia_)
    plt.plot(range(1, 11), wcss)
    plt.title('Elbow Method')
    plt.xlabel('Number of clusters')
    plt.ylabel('WCSS')
    plt.show()

# DBSCAN

In [ ]:
def apply_dbscan(transformed_sample,ellbow_method,cluster,df_name,num_scaler_name,cat_scaler_name,dim_red_method,experiment_name):
    db = DBSCAN(eps=0.1, min_samples=5).fit(transformed_sample)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_
    scatter_plot(transformed_sample,labels) 
    
    X=transformed_sample
    
    # Black removed and is used for noise instead.
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=14)

        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6)

    plt.title('DBSCAN' )
    plt.show()
    print(np.unique(labels))
    return labels

# HDBSCAN

In [ ]:
def apply_hdbscan(transformed_sample,ellbow_method,cluster,df_name,num_scaler_name,cat_scaler_name,dim_red_method,experiment_name):
    hdbscan_labels = hdbscan.HDBSCAN(min_samples=10, min_cluster_size=500).fit_predict(transformed_sample)
    clustered = (hdbscan_labels >= 0)
    plt.scatter(transformed_sample[~clustered, 0],
                transformed_sample[~clustered, 1],
                c=(0.5, 0.5, 0.5),
                s=0.1,
                alpha=0.5)
    plt.scatter(transformed_sample[clustered, 0],
                transformed_sample[clustered, 1],
                c=hdbscan_labels[clustered],
                s=0.1,
                cmap='Spectral');
    
    return hdbscan_labels


## Gaussian 


In [ ]:
def apply_gaussian(df,n_cluster):
    gmm = GaussianMixture(n_components=n_cluster)
    gmm.fit(df)
    proba_lists = gmm.predict_proba(df)
    #Plotting
    colored_arrays = np.matrix(proba_lists)
    colored_tuples = [tuple(i.tolist()[0]) for i in colored_arrays]
    fig = plt.figure(1, figsize=(7,7))
    ax = Axes3D(fig, rect=[0, 0, 0.95, 1], elev=48, azim=134)
    ax.scatter(df[:, 0], df[:, 1], df[:, 2],
              c=colored_tuples, edgecolor="k", s=50)
    ax.set_xlabel("Petal width")
    ax.set_ylabel("Sepal length")
    ax.set_zlabel("Petal length")
    plt.title("Gaussian Mixture Model", fontsize=14)

## Hierachical

In [ ]:
def apply_hierachical(df_dim_red,ellbow_method,n_cluster,df_name,num_scaler_name,cat_scaler_name,dim_red_method,experiment_name):
    Dendogram=False
    #Dendogram: 
    if Dendogram==True:
        try: 
        #load Dendogram image
            plt.figure(figsize=(250, 7))
            experiment_name=experiment_name
            img=mpimg.imread('Cohort/Models/Plots/Dendograms/'+experiment_name+'.png')
            print('Dendogram loaded')
            imgplot = plt.imshow(img)
            plt.show()
        except:
            plt.figure(figsize=(10, 7))
            experiment_name=df_name+'_'+num_scaler_name+'_'+cat_scaler_name+'_'+dim_red_method+'_'+str(dimension)+'_hierachical_'
            plt.title(experiment_name)
            dend = shc.dendrogram(shc.linkage(df_dim_red, method='ward'))  
            plt.savefig('Cohort/Models/Plots/Dendograms/'+experiment_name+'.png')

    # setting distance_threshold=0 ensures we compute the full tree.
    model = AgglomerativeClustering(n_clusters=n_cluster, affinity='euclidean', linkage='ward',distance_threshold=None)

    
    model.fit_predict(df_dim_red)
    scatter_plot(df_dim_red,model.labels_)
   
    
    return model.labels_

# Metrics

## Silhouette Coefficient

In [ ]:
def get_silhouette_Coefficient(labels,df):
    m=metrics.silhouette_score(df, labels, metric='euclidean')
    print('silhouette_score:',m)
    return m
    

## Calinski-Harabasz Index

In [ ]:
def get_calinski_harabasz(labels,df):
    m=metrics.calinski_harabasz_score(df, labels)
    print('Calinski-Harabasz Index:',m)
    return m

## Davies-Bouldin Index

In [ ]:
def get_davies_bouldin(labels,df):
    m=davies_bouldin_score(df, labels)
    print('Davies-Bouldin Index:',m)
    return m

## Feature Analysis


In [ ]:
def analyse_num_column(df,col,kind): 
    total_patient=len(df)
    df_mean=df[col].mean()
    df_std= df[col].std()
    df_median= df[col].median()
    if kind=='Cluster':
        row={'column_name':col,'col_type':'num','cat_total':'','cat_percentage':'','num_mean':df_mean,'num_std':df_std,'num_median':df_median,'total_patient':total_patient}
    else: 
        row={'cat_total_all':'','cat_percentage_all':'','num_mean_all':df_mean,'num_std_all':df_std,'num_median_all':df_median,'total_patient_all':total_patient}
    return row 

In [ ]:
def analyse_cat_column(df,col,kind): 
    total_patient=len(df)
    cat_total=len(df.loc[df[col]==True])
    cat_percentage=(cat_total/total_patient)
    if kind=='Cluster':
        row={'column_name':col,'col_type':'cat','cat_total':cat_total,'cat_percentage':cat_percentage,'num_mean':'','num_std':'','num_median':'','total_patient':total_patient}
    else: 
        row={'cat_total_all':cat_total,'cat_percentage_all':cat_percentage,'num_mean_all':'','num_std_all':'','num_median_all':'','total_patient_all':total_patient}
    return row 

In [ ]:
def analyse_gender_column(df,kind):
    total_patient=len(df)
    cat_total=len(df.loc[df['gender']=='Male'])
    cat_percentage=(cat_total/total_patient)
    if kind=='Cluster':
        row1={'column_name':'gender_male','col_type':'cat','cat_total':cat_total,'cat_percentage':cat_percentage,'num_mean':'','num_std':'','num_median':'','total_patient':total_patient}
    else: 
        row1={'cat_total_all':cat_total,'cat_percentage_all':cat_percentage,'num_mean_all':'','num_std_all':'','num_median_all':'','total_patient_all':total_patient}
    cat_total=len(df.loc[df['gender']=='Female'])
    cat_percentage=(cat_total/total_patient)
    if kind=='Cluster':
        row2={'column_name':'gender_female','col_type':'cat','cat_total':cat_total,'cat_percentage':cat_percentage,'num_mean':'','num_std':'','num_median':'','total_patient':total_patient}
    else: 
        row2={'cat_total_all':cat_total,'cat_percentage_all':cat_percentage,'num_mean_all':'','num_std_all':'','num_median_all':'','total_patient_all':total_patient}
    cat_total=len(df.loc[df['gender']=='Female'])
    return row1, row2 

In [ ]:
def analyse_feature(ctransformer,df_cohort,n_cluster):
    result_array=[]
    min_max_scaler = preprocessing.MinMaxScaler()
    num_columns=ctransformer.transformers[0][2]
    cat_columns=ctransformer.transformers[1][2]
    df_cohort[list(num_columns)] = min_max_scaler.fit_transform(df_cohort[list(num_columns)])
    del cat_columns[:8]
    col=['cat_total_all','cat_percentage_all','num_mean_all','num_std_all','num_median_all','total_patient_all']
    result_all=pd.DataFrame(columns=col)
    for col in num_columns: 
        row=analyse_num_column(df_cohort,col,'all')
        result_all=result_all.append(row, ignore_index=True)
    for col in cat_columns: 
        row=analyse_cat_column(df_cohort,col,'all')
        result_all=result_all.append(row, ignore_index=True)
    row=analyse_gender_column(df_cohort,'all')
    result_all=result_all.append(row[0], ignore_index=True)
    result_all=result_all.append(row[1], ignore_index=True)
    for i in (range(n_cluster)):
        col=['column_name','col_type','cat_total','cat_percentage','num_mean','num_std','num_median','total_patient']
        result=pd.DataFrame(columns=col)
        df=df_cohort.loc[df_cohort[dim_red_method]==i]
        for col in num_columns: 
            row=analyse_num_column(df,col,'Cluster')
            result=result.append(row, ignore_index=True)
        for col in cat_columns: 
            row=analyse_cat_column(df,col,'Cluster')
            result=result.append(row, ignore_index=True)
        row=analyse_gender_column(df,'Cluster')
        result=result.append(row[0], ignore_index=True)
        result=result.append(row[1], ignore_index=True)
        result=pd.concat([result,result_all],axis=1)

        result_array.append(result)
        
    return result_array

In [ ]:
def get_important_features(result_array,n_cluster,top_features):
    for i in (range(n_cluster)):
        test_num=result_array[i].loc[result_array[i]['col_type']=='num']
        test_num['dif_median']=abs(test_num['num_median']-test_num['num_median_all'])
        test_num=test_num.sort_values(by=['dif_median'],ascending=False)
        print('Cluster '+str(i)+' num features \n',test_num[['column_name' ,'num_median','num_median_all']].head(top_features))
        test_cat=result_array[i].loc[result_array[i]['col_type']=='cat']
        test_cat['dif_percentage']=abs(test_cat['cat_percentage']-test_cat['cat_percentage_all'])
        test_cat=test_cat.sort_values(by=['dif_percentage'],ascending=False)
        print('Cluster '+str(i)+' cat features \n',test_cat[['column_name' ,'cat_percentage','cat_percentage_all']].head(top_features))

 # Anova 

In [ ]:
def num_feature_importance_anova(df,ctransformer,dim_red_method,n_cluster,top_features):
    df_temp=df
    #replace cluster names 
    for cluster in (range(n_cluster)):
        cluster_name='cluster_'+str(cluster)
        df[dim_red_method].replace({cluster: cluster_name},inplace=True)
    #normalize num columns
    min_max_scaler = preprocessing.MinMaxScaler()
    num_columns=ctransformer.transformers[0][2]
    df_temp[list(num_columns)] = min_max_scaler.fit_transform(df_temp[list(num_columns)])
    #iterate over num columns and calculate the p-Value: 
    col=['column name','F-Value','p-value','absolute_p','compared to other']
    result_all=pd.DataFrame(columns=col) 
    result_anova=[]
    for cluster in df_temp[dim_red_method].unique():
        result_all=pd.DataFrame(columns=col)
        df_temp['temp_cluster']=df_temp[dim_red_method]
        df_temp.loc[df[dim_red_method] != cluster, "temp_cluster"] = "other_cluster"
        for num_col in num_columns: 
            feature=num_col
            result = df_temp.groupby('temp_cluster')[feature].apply(list)
            #print(result)
            feature_value_1=result[cluster]
            #print(feature_value_1)
            feature_value_2=result['other_cluster']
            mean_1=mean(feature_value_1)
            mean_2=mean(feature_value_2)
            if mean_1 > mean_2: 
                compared='higher'
            else:
                compared='lower'
            #print(len(result['cluster_3']))
            #print(len(result['cluster_0']))
            F, p = stats.f_oneway(*result)
            p=format(p, '.300000000g')
            p=float(p)
            if p!=0:
                importance=abs(np.log(p))
            else: 
                importance=0
            row={'column name':(feature+'_'+cluster),'F-Value':F,'p-value':p,'absolute_p':importance,'compared to other':compared}
            result_all=result_all.append(row, ignore_index=True)
        result_all=result_all.sort_values(by=['absolute_p'],ascending=False)
        result_anova.append(result_all)
    #result_all=result_all.drop_duplicates(subset='column name',keep='first', inplace=False)
    #return result_all.head(top_features)
    return result_anova

# Chi Test

In [ ]:
#https://www.pythonfordatascience.org/chi-square-test-of-independence-python/
def cat_feature_importance(df,ctransformer,sup_colums,dim_red_method,n_cluster,top_features):
    #replace cluster names 
    #establish two categories in all Categories 
    
   
    for cluster in (range(n_cluster)):
        cluster_name='cluster_'+str(cluster)
        df[dim_red_method].replace({cluster: cluster_name},inplace=True)
    df=df.replace(True, 'Yes')
    df=df.replace(False,'No')
    df=df.fillna('No')
    df=df.replace(1, 'Yes')
    df=df.replace(0,'No')
    df=df.fillna('No')
    col=['column name','Pearson Chi-square','Cramers V','p-value','absolute_p','compared to other']
    result_all=pd.DataFrame(columns=col)
    result_chi=[]
    for cluster in df[dim_red_method].unique():
        result_all=pd.DataFrame(columns=col)
        df['temp_cluster']=df[dim_red_method]
        df.loc[df[dim_red_method] != cluster, "temp_cluster"] = "other_cluster"
        #print(df[[dim_red_method,'temp_cluster']])     
        cat_columns=ctransformer.transformers[1][2]
        #iterate over cat columns and calculate the p-Value: 
        for cat_col in cat_columns: 
            feature=cat_col
            crosstab, test_results, expected = rp.crosstab(df[feature], df['temp_cluster'],
                                                   test= "chi-square",
                                                   expected_freqs= True,
                                                   prop= "cell")
            p=format(test_results["results"][1], '.300000000g')
            #print(p)
           # if test_results["results"][1]!=0:
            p=float(p)
            if p!=0:

                importance=abs(np.log(p))
            else: 
                importance=0
            compared=''
            if feature !='gender':
                feature_count_1=len(df.loc[df['temp_cluster']==cluster])
                feature_cluster=df.loc[df['temp_cluster']==cluster]
                feature_percentage_1=(len(feature_cluster.loc[feature_cluster[feature]=='Yes'])/feature_count_1)
                #print(feature_percentage_1)
    
                feature_count_2=len(df.loc[df['temp_cluster']=='other_cluster'])
                feature_cluster_2=df.loc[df['temp_cluster']=='other_cluster']
                feature_percentage_2=(len(feature_cluster_2.loc[feature_cluster_2[feature]=='Yes'])/feature_count_2)
                #print(feature_percentage_2)
                if feature_percentage_1 > feature_percentage_2: 
                    compared='higher'
                else:
                    compared='lower'
            row={'column name':(feature+'_'+cluster),'Pearson Chi-square':test_results["results"][0],'Cramers V':test_results["results"][2],'p-value':p,'absolute_p':importance,'compared to other':compared}
            #row={'column name':feature,'Pearson Chi-square':test_results["results"][0],'Cramers V':test_results["results"][2],'p-value':p,'absolute_p':importance}
            result_all=result_all.append(row, ignore_index=True)
        for cat_col in sup_colums: 
            #print('Calculaint Supervised features')
            feature=cat_col
            crosstab, test_results, expected = rp.crosstab(df[feature], df['temp_cluster'],
                                                   test= "chi-square",
                                                   expected_freqs= True,
                                                   prop= "cell")
            #print(crosstab)
            p=format(test_results["results"][1], '.300000000g')
            #print(p)
           # if test_results["results"][1]!=0:
            p=float(p)
            if p!=0:

                importance=abs(np.log(p))
            else: 
                importance=0
            compare=''
            if feature !='gender':
                feature_count_1=len(df.loc[df['temp_cluster']==cluster])
                feature_cluster=df.loc[df['temp_cluster']==cluster]
                feature_percentage_1=(len(feature_cluster.loc[feature_cluster[feature]=='Yes'])/feature_count_1)
               # print(feature_percentage_1)
    
                feature_count_2=len(df.loc[df['temp_cluster']=='other_cluster'])
                feature_cluster_2=df.loc[df['temp_cluster']=='other_cluster']
                feature_percentage_2=(len(feature_cluster_2.loc[feature_cluster_2[feature]=='Yes'])/feature_count_2)
               # print(feature_percentage_2)
                if feature_percentage_1 > feature_percentage_2: 
                    compared='higher'
                else:
                    compared='lower'
            row={'column name':(feature+'_'+cluster),'Pearson Chi-square':test_results["results"][0],'Cramers V':test_results["results"][2],'p-value':p,'absolute_p':importance,'compared to other':compared}
            #row={'column name':feature,'Pearson Chi-square':test_results["results"][0],'Cramers V':test_results["results"][2],'p-value':p,'absolute_p':importance}
            result_all=result_all.append(row, ignore_index=True)
        result_all=result_all.sort_values(by=['absolute_p'],ascending=False)
        result_chi.append(result_all)
    #result_all=result_all.drop_duplicates(subset='column name',keep='first', inplace=False)
    #return result_all.head(top_features)
    return result_chi

In [ ]:
#with open('Cohort/Models/ColumnTransformer/'+df_name+'_'+num_scaler_name+'_'+cat_scaler_name+'.pkl', 'rb') as f:
#            ctransformer = pickle.load(f)
#cat_columns=ctransformer.transformers[1][2]
#cat_columns

# T Test

In [ ]:
def num_feature_importance_t_test(df,ctransformer,dim_red_method,n_cluster,top_features,inp_colums,merge_w_inpatient):
    df_temp=df
    #replace cluster names 
    for cluster in (range(n_cluster)):
        cluster_name='cluster_'+str(cluster)
        df[dim_red_method].replace({cluster: cluster_name},inplace=True)
    #normalize num columns
    min_max_scaler = preprocessing.MinMaxScaler()
    num_columns=ctransformer.transformers[0][2]
    if merge_w_inpatient==True:
        inpatient=inp_colums[0]
        num_columns.append(inpatient)
    #print(num_columns)
    df_temp[list(num_columns)] = min_max_scaler.fit_transform(df_temp[list(num_columns)])
    #iterate over num columns and calculate the p-Value: 
    col=['column name','T-Statistics','p-value','absolute_p','compared to other']
    result_all=pd.DataFrame(columns=col) 
    result_t_test=[]
    for cluster in df_temp[dim_red_method].unique():
        result_all=pd.DataFrame(columns=col)
        df_temp['temp_cluster']=df_temp[dim_red_method]
        df_temp.loc[df[dim_red_method] != cluster, "temp_cluster"] = "other_cluster"
        for num_col in num_columns: 
            feature=num_col
            feature_value_1=df_temp.loc[df_temp['temp_cluster']==cluster][feature].values
            feature_value_2=df_temp.loc[df_temp['temp_cluster']=="other_cluster"][feature].values
            statistics,p=stats.ttest_ind(feature_value_1, feature_value_2, equal_var = False)
            mean_1=feature_value_1.mean()
            mean_2=feature_value_2.mean()
            if mean_1 > mean_2: 
                compared='higher'
            else:
                compared='lower' 
           # print(feature_value_1)
           # print(feature_value_2)
           # print(p)
            p=format(p, '.300000000g')
            p=float(p)
            if p!=0:
                importance=abs(np.log(p))
            else: 
                importance=0
            row={'column name':(feature+'_'+cluster),'T-Statistics':statistics,'p-value':p,'absolute_p':importance,'compared to other':compared}
            result_all=result_all.append(row, ignore_index=True)
        result_all=result_all.sort_values(by=['absolute_p'],ascending=False)
        result_t_test.append(result_all)        
    return result_t_test
     

# Generic Cluster information: 

In [ ]:
#function for statistics: 
def get_base_characteristic_value(df , characteristic , kind):    
    if kind=="mean": 
        df_mean=df[characteristic].mean()
        df_std= df[characteristic].std()
        df_max= df[characteristic].max()
        df_min= df[characteristic].min()
        if characteristic == "HF_Onset_age_in_days":
            base_characteristics_cohort=pd.DataFrame({'Variable': [characteristic+"_mean", characteristic+"_std", characteristic+"_max", characteristic+"_min"],
                                                  'Value': [(df_mean/365), (df_std/365), (df_max/365), (df_min/365)],})
        else:
            base_characteristics_cohort=pd.DataFrame({'Variable': [characteristic+"_mean", characteristic+"_std", characteristic+"_max", characteristic+"_min"],
                                                  'Value': [(df_mean), (df_std), (df_max), (df_min)],})
        
    if kind=="count":
        base_characteristics_cohort=pd.DataFrame(columns=["Variable","Value"])
        feature_value=df[characteristic].unique()
        #print(feature_value)
        for value in feature_value: 
            df_condition=df.loc[df[characteristic]==value]
            df_percent= df_condition.shape[0]/df.shape[0]
            #print(df_percent)
            new_row1 = {'Variable': value+"_total",'Value': df_condition.shape[0]}
            new_row2 = {'Variable': value+"_relation",'Value': df_percent}
            base_characteristics_cohort=base_characteristics_cohort.append(new_row1, ignore_index=True)
            base_characteristics_cohort=base_characteristics_cohort.append(new_row2, ignore_index=True)
       # print(df_condition.shape[0], df_percent)
    #print (base_characteristics_cohort)
    return base_characteristics_cohort

def get_base_characteristics(df, characteristics): 
    base_characteristics_cohort=pd.DataFrame(columns=["Variable","Value"])
    for characteristic in characteristics:
        intermediate_base_characteristics_cohort=get_base_characteristic_value(df,characteristic[0],characteristic[1])
        base_characteristics_cohort=pd.concat([base_characteristics_cohort,intermediate_base_characteristics_cohort])
    #print(base_characteristics_cohort)
    return base_characteristics_cohort

def get_cluster_information(df,dim_red_method,base_characteristics):
    
    baseline_characteristics=[]
    for cluster in df[dim_red_method].unique(): 
        cluster_characteristics=[]
        df_temp=df.loc[df[dim_red_method] == cluster]
        df_base_characteristics=get_base_characteristics(df_temp, base_characteristics)
        
        cluster_characteristics.append(cluster)
        cluster_characteristics.append(len(df_temp))
        cluster_characteristics.append(df_base_characteristics)
        baseline_characteristics.append(cluster_characteristics)
    return baseline_characteristics
def get_cluster_statistics(df,dim_red_method):
    #load inpatient and EF dataframe 
    hospitalization = pq.read_table('Cohort/Feature_Extraction/days_in_hospital.parquet').to_pandas()
    ef=pq.read_table('Cohort/Feature_Extraction/avg_EF.parquet').to_pandas()
    #merge both to the df: 
    df_cohort=pd.merge(df, hospitalization, how='left', left_index=True, right_on='medical_record_number')
    df_cohort=pd.merge(df_cohort, ef, how='left',left_index=True, right_on='medical_record_number')
    #get average days in hospital per patient per cluster
    base_characteristics=[
        [ "avg_ef","mean"],
        ["days_in_hospital","mean"],
        [ "HF_Onset_age_in_days","mean"],
        ["gender","count"]
        ]
    baseline_characteristics=get_cluster_information(df_cohort,dim_red_method,base_characteristics)
    print (baseline_characteristics)
    df_boxplt=df_cohort[["avg_ef",dim_red_method]]
    df_boxplt.boxplot(by=dim_red_method)
    df_boxplt=df_cohort[[ "days_in_hospital",dim_red_method]]
    df_boxplt.boxplot(by=dim_red_method)                   
   

    return baseline_characteristics
        #print(str(cluster))
        #print(len(df_temp))
        
        #print(df_temp_baseline)

# Visualization 

In [ ]:
def subStringCluster(string):
    a_string=string
    split_string=a_string.split('_cluster_',1)
    substring = split_string[0]
    return substring

In [ ]:
def create_overview_table(conv_df,features_tuples,features,dim_red_method): 
    feature_evaluation_df = pd.DataFrame()
    feature_evaluation_df['features']=features
    #print (feature_evaluation_df)
    for cluster in conv_df[dim_red_method].unique(): 
        feature_evaluation_df[cluster]=0
        cluster_df= conv_df.loc[conv_df[dim_red_method]==cluster]        
        for features_tuple in features_tuples:
            if features_tuple[1]=='categorical':
                sum_feature=cluster_df[features_tuple[0]].sum()
                percentage=sum_feature/len(cluster_df)
                feature_evaluation_df.loc[feature_evaluation_df['features']==features_tuple[0],cluster]=percentage
                
                #print('categorical')
            if features_tuple[1]=='numeric':
                mean_feature=cluster_df[features_tuple[0]].mean()
                median_feature=cluster_df[features_tuple[0]].median()
                feature_evaluation_df.loc[feature_evaluation_df['features']==features_tuple[0],cluster]=str((str(mean_feature)+'/'+str(median_feature)))
                #print('numeric') '''
   # print(feature_evaluation_df)
    return feature_evaluation_df

In [ ]:
def getTopCluster(evaluation_pandas, n_topFeature, n_cluster ): 
    topFeatures=[]
    for n in range(n_cluster):
        #print(n)
        features=[]
         #categorical features
        features=evaluation_pandas[2][n]['column name'].values
        all_features = evaluation_pandas[2][n]
        x=0
        for i in range(n_topFeature):
            feature=subStringCluster(features[x])
            
            if 'Procedure' in feature: 
               # print (feature)
                #x=x+1
                #print(subStringCluster(features[x]))
                #topFeatures.append(subStringCluster(features[x]))
                i=i-1
            elif feature != 'gender' :
                f=all_features.loc[all_features['column name']==features[x]]
                p_value=f['p-value'].values
                if p_value < 0.05 and p_value!=0.0 :
                    topFeatures.append([subStringCluster(features[x]),'categorical'])
                    #print(feature)
                
            else: 
                i=i-1
            x=x+1
        
        #numeric
        features=evaluation_pandas[1][n]['column name'].values
        all_features = evaluation_pandas[1][n]
        for i in range(n_topFeature):
            f=all_features.loc[all_features['column name']==features[i]]
            p_value=f['p-value'].values
            if p_value < 0.05 and p_value!=0.0 :
                topFeatures.append([subStringCluster(features[i]),'numeric'])
    topFeatures_tuple=set(tuple(t)for t in topFeatures)
    #print(topFeatures_tuple)
    topFeatures=[t[0] for t in topFeatures_tuple]
    #print(topFeatures)
    #topFeatures=set(topFeatures)
    #topFeatures=list(topFeatures)
    #print(topFeatures)
    return topFeatures_tuple, topFeatures
        

In [ ]:
#https://github.com/hpi-dhc/robotehr/blob/e3673aef701aa817c74d04170986f01fa191212a/robotehr/evaluation/risk_groups.py#L70-L100
def plot_risk_groups(df, features,dim_red_method, friendly_names_converter=None, filename='', nrows=2, figsize=[12,3]):
    #features=features[:2]
    #ncols = int(len(features) / nrows)
    ncols=1
    #fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    fig, ax = plt.subplots(len(features),figsize=figsize)
    fig.tight_layout(pad=3.0)
    print(len(features))
    for i in range(len(features)):
        #row_index = int(i / ncols)
        row_index=i
        #col_index = i % int(len(features) / nrows)
        col_index=0

        #current_axis = ax[row_index][col_index] if nrows > 1 else ax[col_index]
        current_axis = ax[row_index]
        if df[features[i]].min() == 0 and df[features[i]].max() == 1:
            current_axis.set_ylim(bottom=-0.5, top=1.5)
        sns.violinplot(
            x=dim_red_method,
            y=features[i],
            data=df,
            palette="muted",
            ax=current_axis,
            hue='gender'
        )
        if friendly_names_converter:
            title = friendly_names_converter.get(features[i])
        else:
            title = features[i]
        if len(title) > 50:
            title = f'{title[:50]} ...'
        current_axis.set_title(f'{title}', fontsize=20)
        current_axis.set_xlabel('')
        current_axis.set_ylabel('')
    if filename:
        fig.savefig(filename, dpi=300, bbox_inches="tight")
    return fig

In [ ]:
def map_feature_names(feature_evaluation):
    feature_look_up=pq.read_table('Cohort/feature_look_up.parquet').to_pandas()
    feature_evaluation['human_readable']=''
    for index,r in feature_evaluation.iterrows():
        lookuplist=feature_look_up['original_feature_name'].to_list()
        if r['features'] in lookuplist:
            human_readable_row=feature_look_up.loc[feature_look_up['original_feature_name']==r['features']]
            human_readable=human_readable_row['human_readable'].values
            #print(human_readable)
            feature_evaluation.loc[feature_evaluation['features']==r['features'],'human_readable']=human_readable[0]
        else :
            feature_evaluation.loc[feature_evaluation['features']==r['features'],'human_readable']=r['features']
    return feature_evaluation

In [ ]:
def plotTopFeatures(df_path,df,merge_w_supervised,dim_red_method, evaluation_results,  n_cluster, n_topFeatures): 
    '''#convert the dataframe
    df_origin=pq.read_table(df_path).to_pandas()
    #print(df_origin['gender'])
    df_origin[dim_red_method]=df[dim_red_method]
    conv_df=df_origin
    if merge_w_supervised==True:
        df_supervised_merge= pq.read_table('Cohort/Feature_Extraction/ALL_HF_cohort_supervised_only_ever_diag_drugFORMerge.parquet').to_pandas()
        conv_df.index = conv_df.index.map(str)
        df_supervised_merge.index = df_supervised_merge.index.map(str)
        conv_df=pd.merge(conv_df, df_supervised_merge, left_on='medical_record_number', right_on='medical_record_number')
     '''
    conv_df=pq.read_table(df_path).to_pandas()
    if merge_w_supervised==True:
            df_supervised_merge= pq.read_table('Cohort/Feature_Extraction/ALL_HF_cohort_supervised_only_ever_diag_drugFORMerge_wLab.parquet').to_pandas()
            conv_df.index = conv_df.index.map(str)
            df_supervised_merge.index = df_supervised_merge.index.map(str)
            conv_df=pd.merge(conv_df, df_supervised_merge, left_on='medical_record_number', right_on='medical_record_number')
        
    conv_df=conv_df.replace(True, 1)
    conv_df=conv_df.replace(False,0)
    conv_df=conv_df.replace('yes', 1)
    conv_df=conv_df.replace('no',0)
    conv_df=conv_df.fillna(0)
    conv_df[dim_red_method]=df[dim_red_method]
    conv_df=conv_df.sort_values(by=[dim_red_method],ascending=True)
    #get top featrues: 
    evaluation_pandas=evaluation_results
    features_tuples,features=getTopCluster(evaluation_pandas, n_topFeatures, n_cluster)
    #plot features 
    #print (cluster_name)
   # fig_x=12*len(features)
    fig_y=8*len(features)
    plot_risk_groups(conv_df, features, dim_red_method,friendly_names_converter=None, filename='', nrows=10, figsize=[12,fig_y])
    feature_evaluation_df=create_overview_table(conv_df,features_tuples,features,dim_red_method)
    feature_evaluation_df=map_feature_names(feature_evaluation_df)
    return feature_evaluation_df
    
    

# Nice Scatter Plot 

In [ ]:
def scatter_plot(df,labels):
    sns.set(style='white', rc={'figure.figsize':(10,8)})
    sns.color_palette("Set2")
    plt.scatter(df[:, 0], df[:, 1], c=labels, s=0.1, cmap='Accent');
    plt.show()
   # px.scatter(df[:, 0], df[:, 1], c=labels, s=0.1 ,color_continuous_scale=px.colors.sequential.Inferno);
    #px.show()

# pipeline 

In [ ]:
def cluster_pipeline(df_path,df_name,age_filter,drop_gender,drop_age,tune_umap,umap_distance,umap_neighbors, check_pca,plotting,num_scaler_name,cat_scaler_name, dim_red_method,dimension,a_f_decoder,a_f_encoder,batchsize,epochs,optimizer,loss_function,cluster_method,ellbow_method,n_cluster,anova,chi,top_features,merge_w_supervised,merge_w_inpatient):
    experiment_name=df_name+'_'+num_scaler_name+'_'+cat_scaler_name
    if drop_gender==True:
        experiment_name=experiment_name+'_woGender'
        print(experiment_name)
    if drop_age==True:
        experiment_name=experiment_name+'_woAge'
        print(experiment_name)
    labels=[]
    df_origin= pq.read_table(df_path).to_pandas()
     #Age Filter:
    if age_filter==True: 
        df_origin.loc[(df_origin['HF_Onset_age_in_days'] > 32850),'HF_Onset_age_in_days']=32850
        #print(df_cohort)
        
    #general columns that should be not included in the clustering    
    col_for_dropping=[
        'religion',
        'race',
        'patient_ethnic_group',
        'deceased_indicator',
        'mother_account_number',
        'address_zip',
        'marital_status_code']
    
    #Exclude gender in Cluster analysis
    if drop_gender==True: 
        col_for_dropping.append('gender')
    #Exclude age from Cluster Analysis
    if drop_age==True: 
        col_for_dropping.append('HF_Onset_age_in_days')
    df_cohort=df_origin.drop(col_for_dropping,axis=1)
    
    #print(df_cohort)
    #ColumnTransformer df,df_name,num_scaler_name,cat_scaler_name
    a=apply_columnTransformer(df_cohort,df_name,num_scaler_name,cat_scaler_name,experiment_name)
    transformed_df= a[0]
    ctransformer=a[1]
    loss=0
    n_layer=0
    # test best PCA Dimension: 
    if check_pca==True:
        pca = PCA().fit(transformed_df)
        fig, ax = plt.subplots()
        d_pca=np.cumsum(pca.explained_variance_ratio_)
        #x="year", y="passengers"
        #sns.set(style='white', rc={'figure.figsize':(12,10)})
        g=sns.lineplot(data=d_pca,ax=ax)
        g.set_xticklabels([0,25,50,75,100,125,150,250])
        #g.set_yticklabels([0,0.25,0.50,0.75,1])
        ax.set_xlim(0,300)
        ax.set_ylim(0,1)
        #ax.set_xticks(range(1,200))
        plt.show()
        #print(len(d))
        #sns.lineplot(data=may_flights, x="year", y="passengers")
        #sns.set(style='white', rc={'figure.figsize':(10,8)})
        #fig,ax=plt.plot(np.cumsum(pca.explained_variance_ratio_))
        #ax.set_xticks(range(1,250))
        #plt.xlabel('number of components')
        #plt.ylabel('cumulative explained variance');
    #return True
    #Dimension Reduction: 
    experiment_name=experiment_name+'_'+dim_red_method+'_'+str(dimension)
    if tune_umap==True: 
        experiment_name=experiment_name+'_'+str(umap_distance)+'_'+str(umap_neighbors)
    try: 
        if dim_red_method=='AE':
            n_layer=3
            load = np.load('Cohort/Models/DimReduction/'+df_name+'_'+num_scaler_name+'_'+cat_scaler_name+'_AE_'+str(dimension)+'_'+a_f_encoder+'_'+a_f_decoder+'_'+str(n_layer)+'_'+str(batchsize)+'_'+str(epochs)+'_'+optimizer+'_'+loss_function+'.npz')
            df_dim_red=load['a']
            print('df_dim_red loaded')
        else:            
            load = np.load('Cohort/Models/DimReduction/'+experiment_name+'.npz')
            df_dim_red=load['a']
            print('df_dim_red loaded!!!')
    except:  
        if dim_red_method=="PPCA": 
            df_dim_red=apply_ppca(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="TSNE": 
            df_dim_red=apply_TSNE(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="ICA": 
            df_dim_red=apply_ICA(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="SVD": 
            df_dim_red=apply_svd(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="PCA": 
            df_dim_red=apply_pca(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="IPCA": 
            df_dim_red=apply_ipca(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="KPCA": 
            df_dim_red=apply_kpca(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="LDA": 
            df_dim_red=apply_lda(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="PPCA_TSNE":
            df_dim_red=apply_ppca(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
            df_dim_red=apply_TSNE(df_cohort,df_dim_red,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="PCA_TSNE":
            df_dim_red=apply_pca(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
            df_dim_red=apply_TSNE(df_cohort,df_dim_red,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="ICA_TSNE":
            df_dim_red=apply_ICA(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
            df_dim_red=apply_TSNE(df_cohort,df_dim_red,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="KPCA_TSNE":
            df_dim_red=apply_kpca(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
            df_dim_red=apply_TSNE(df_cohort,df_dim_red,dimension,df_name,num_scaler_name,cat_scaler_name,experiment_name)
        if dim_red_method=="UMAP":
            df_dim_red=apply_umap(df_cohort,transformed_df,dimension,umap_distance,umap_neighbors,df_name,num_scaler_name,cat_scaler_name,experiment_name,True)
        if dim_red_method=='AE':
            n_layer=3
            df_dim_red,loss=apply_AE(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,a_f_decoder,a_f_encoder,n_layer,batchsize,epochs,optimizer,loss_function)
        if dim_red_method=='AE_TSNE':
            n_layer=3
            df_dim_red,loss=apply_AE(df_cohort,transformed_df,dimension,df_name,num_scaler_name,cat_scaler_name,a_f_decoder,a_f_encoder,n_layer,batchsize,epochs,optimizer,loss_function)
            df_dim_red=apply_TSNE(df_cohort,df_dim_red,dimension,df_name,num_scaler_name,cat_scaler_name)
        if dim_red_method=="":
            df_dim_red=transformed_df
        if dim_red_method=='AE': 
            np.savez_compressed('Cohort/Models/DimReduction/'+df_name+'_'+num_scaler_name+'_'+cat_scaler_name+'_AE_'+str(dimension)+'_'+a_f_encoder+'_'+a_f_decoder+'_'+str(n_layer)+'_'+str(batchsize)+'_'+str(epochs)+'_'+optimizer+'_'+loss_function+'.npz',a=df_dim_red)
        else:
            np.savez_compressed('Cohort/Models/DimReduction/'+experiment_name+'.npz',a=df_dim_red)
    
    
   #extend the experiment_name
    experiment_name=experiment_name+'_'+cluster_method+'_'+str(n_cluster)
    if cluster_method=="kmeans":
        labels=apply_kmeans(df_dim_red,ellbow_method,n_cluster,df_name,num_scaler_name,cat_scaler_name,dim_red_method,experiment_name)
    if cluster_method=="gaussian": 
        apply_gaussian(df_dim_red,4)
    if cluster_method=="hierarchical":
        labels=apply_hierachical(df_dim_red,ellbow_method,n_cluster,df_name,num_scaler_name,cat_scaler_name,dim_red_method,experiment_name)
    if cluster_method=="dbscan":
        labels=apply_dbscan(df_dim_red,ellbow_method,n_cluster,df_name,num_scaler_name,cat_scaler_name,dim_red_method,experiment_name)
    if cluster_method=="hdbscan":
        labels=apply_hdbscan(df_dim_red,ellbow_method,n_cluster,df_name,num_scaler_name,cat_scaler_name,dim_red_method,experiment_name)
    
    if plotting==True:      
        #prepare data for plotting 
        df_dim_red_plot=apply_umap(df_cohort,df_dim_red,2,umap_distance,umap_neighbors,df_name,num_scaler_name,cat_scaler_name,experiment_name,False)        
        #print first 2 dim of dimensionality reduced data:
        scatter_plot(df_dim_red_plot,None)
        scatter_plot(df_dim_red_plot,labels)
    
   # evaluation_results=[]
    if len(labels)!=0: 
        evaluation_results=pq.read_table('Cohort/Models/Metrics_Results.parquet').to_pandas()
        print(experiment_name)
        if experiment_name in evaluation_results.values:
            t=evaluation_results.loc[evaluation_results['Experiment Name'] == experiment_name]
            print(t)
        else :
            print(labels)
            silhouette_Coefficient= get_silhouette_Coefficient(labels,df_dim_red)
            calinski_harabasz=get_calinski_harabasz(labels,df_dim_red)
            davies_bouldin=get_davies_bouldin(labels,df_dim_red)
            if dim_red_method!='PPCA'and dim_red_method!='PPCA_TSNE':
                silhouette_Coefficient_original_Cohort= get_silhouette_Coefficient(labels,transformed_df)
                calinski_harabasz_original_Cohort=get_calinski_harabasz(labels,transformed_df)
                davies_bouldin_original_Cohort=get_davies_bouldin(labels,transformed_df)
            else: 
                silhouette_Coefficient_original_Cohort=0
                calinski_harabasz_original_Cohort=0
                davies_bouldin_original_Cohort=0
            evaluation_results=evaluation_results.append({'Experiment Name':experiment_name,'Dataset':df_name,'Numerical Scaler':num_scaler_name,'Categorical Scaler':cat_scaler_name,'Dimension Reduction Method':dim_red_method,'Number of Dimension':dimension,'Activation Function Decoder':a_f_decoder,'Activation Function Encoder':a_f_encoder,'Number of Layer':n_layer,'batchsize':str(batchsize),'epochs':str(epochs),'optimizer':optimizer,'loss function':loss_function,'validation loss':loss,'Cluster Method':cluster_method,'Number of Cluster':n_cluster,'silhouette_Coefficient':silhouette_Coefficient , 'calinski_harabasz':calinski_harabasz , 'davies_bouldin':davies_bouldin,'silhouette_Coefficient_original_Cohort':silhouette_Coefficient_original_Cohort , 'calinski_harabasz_original_Cohort':calinski_harabasz_original_Cohort , 'davies_bouldin_original_Cohort':davies_bouldin_original_Cohort} , ignore_index=True)
            evaluation_results.to_parquet('Cohort/Models/Metrics_Results.parquet')
        df_cohort[dim_red_method]=labels
    
        #result_array=analyse_feature(ctransformer,df_cohort,n_cluster)
        #get_important_features(result_array,n_cluster,40)
        #Test try to add the supervised features: 
        sup_colums=[]
        inp_colums=[]
        if merge_w_supervised==True:
            df_supervised_merge= pq.read_table('Cohort/Feature_Extraction/ALL_HF_cohort_supervised_only_ever_diag_drugFORMerge_wLab.parquet').to_pandas()
            df_cohort.index = df_cohort.index.map(str)
            df_supervised_merge.index = df_supervised_merge.index.map(str)
            sup_colums=df_supervised_merge.columns
            df_cohort=pd.merge(df_cohort, df_supervised_merge, left_on='medical_record_number', right_on='medical_record_number')
        if merge_w_inpatient==True:
            df_inpatient_merge= pq.read_table('Cohort/Feature_Extraction/Supervised_ALL_HF/inpatient_events_merge_wLab.parquet').to_pandas()
            df_cohort.index = df_cohort.index.map(str)
            df_inpatient_merge.index = df_inpatient_merge.index.map(str)
            inp_colums=df_inpatient_merge.columns
            df_cohort=pd.merge(df_cohort, df_inpatient_merge, left_on='medical_record_number', right_on='medical_record_number')
            print(df_cohort)
        #else: 
           # sup_colums=[]
            #inp_colums=[] 
        df_cohort.to_parquet('Cohort/Models/Cluster/'+experiment_name+'.parquet')
        #print(df_cohort)
        evaluation_results=[]
        df_origin[dim_red_method]=df_cohort[dim_red_method]
        #added for without gender: NOT NEEEEDED
        df_cohort['gender']=df_origin['gender']
        df_cohort['HF_Onset_age_in_days']=df_origin['HF_Onset_age_in_days']
        cluster_information=get_cluster_statistics(df_cohort,dim_red_method)
        evaluation_results.append(cluster_information)
        if anova==True: 
            top_numerical_features_anova=num_feature_importance_anova(df_cohort,ctransformer,dim_red_method,n_cluster,top_features)
            print('Top Numerical features: \n',top_numerical_features_anova)
            evaluation_results.append(top_numerical_features_anova)
        
        if t_test==True: 
            top_numerical_features_t_test=num_feature_importance_t_test(df_cohort,ctransformer,dim_red_method,n_cluster,top_features,inp_colums,merge_w_inpatient)
            print('Top Numerical features: \n',top_numerical_features_t_test)
            evaluation_results.append(top_numerical_features_t_test)   
        if chi==True: 
            top_catigorical_features=cat_feature_importance(df_cohort,ctransformer,sup_colums,dim_red_method,n_cluster,top_features)
            print('Top Categorical features: \n',top_catigorical_features)
            evaluation_results.append(top_catigorical_features)
        for entry in evaluation_results: 
            print(entry)
        np.savez_compressed('Cohort/Models/ClusterEvaluation/'+experiment_name+'_evaluation.npz', a=evaluation_results)
    return df_cohort,evaluation_results

# Pipieline Configutations 

## Dataframe
- df_path: Path to dataframe (String)
- df_name: Name of dataframe (String)
- age_filter: Age over 90 is fixed to 90 (Boolean)
- drop_age: age will be not considered in the pipeline (Boolean)
- drop_gender: gender will be not considered in the pipeline (Boolean)
## Preprocessing
- scaler: Encoder for Categorical Columns:
    - num_scaler_name: 
        - StandardScaler
        - MinMaxScaler
    - cat_scaler_name:
        - BinaryEncoder 
        - OneHotEncoder
## Dimension Reduction Methods
- dim_red_method:
    - PPCA
    - ICA
    - PCA
        - check_pca: Calculating the Variance represented by the diffreent numbers of dimensions(Boolean)
    - KPCA
    - TSNE
    - SVD
    - LDA
    - PCA_TSNE
    - ICA_TSNE
    - AE
         - a_f_decoder: Activation Function of the decoder 
         - a_f_encoder: Activation Function of the encoder 
         - batchsize
         - epochs
             -optimizer
          - loss_function
    - AE_TSNE
    - UMAP
        - tune_umap: different configurations are tried out (Boolean)
        - umap_distace: Minimum Distance between the data points (Float)
        - umap_neighbours: Number of Neighbours (Float)
    
- dimension: number of dimensions the dataset should be reduced to 
## Clustering
- cluster_method: 
    - kmenas
    - hierarchical (AgglomerativeClustering)
- ellbow-method: True or false
- n_cluster: number of cluster that should be applied to the dataset
## Feature Evaluation
- anova: apply anova test on numerical features
- chi: apply chi test on categorical features
- top_features: Number of top features that should be printed out 
 
## General
- plotting: Plotting of Scatter plots (Boolean)
     


## Configuration of CLuster Pipeline example:

In [ ]:
df_path='Cohort/Feature_Extraction/ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab.parquet'
df_name='ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab'
#df_path='Cohort/Feature_Extraction/ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_small_cleaned.parquet'
#df_name='ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_small_cleaned'
#_wSupervised
#df_path='Cohort/Feature_Extraction/ALL_HF_cohort_supervised_only_ever_diag_drug.parquet'
#df_name='ALL_HF_cohort_supervised_only_ever_diag_drug.parquet'
age_filter=True
drop_gender=True
drop_age=True
tune_umap=False
umap_distance=0.1
umap_neighbors=50
check_pca=False
plotting=True
num_scaler_name="MinMaxScaler"
cat_scaler_name='BinaryEncoder'
dim_red_method='UMAP'# 'ICA_TSNE'
a_f_decoder=''
a_f_encoder=''
batchsize=''
epochs=''
optimizer=''
loss_function=''
cluster_method='kmeans'#'hierarchical'
ellbow_method=False
n_cluster=3
dimension=70
anova=False
t_test=True
chi=True
top_features=40
merge_w_supervised=True
merge_w_inpatient=False
df,evaluation_results=cluster_pipeline(df_path,df_name,age_filter,drop_gender,drop_age,tune_umap,umap_distance,umap_neighbors, check_pca,plotting,num_scaler_name,cat_scaler_name, dim_red_method,dimension,a_f_decoder,a_f_encoder,batchsize,epochs,optimizer,loss_function,cluster_method,ellbow_method,n_cluster,anova,chi,top_features,merge_w_supervised,merge_w_inpatient)

## Feature Evaluation example 

In [ ]:
feature_evaluation_df=plotTopFeatures(df_path,df,merge_w_supervised,dim_red_method, evaluation_results,n_cluster ,5)

In [ ]:
feature_evaluation_df

In [ ]:
feature_evaluation_df.to_excel("Cohort/Models/Feature_evaluation_"+dim_red_method+str(dimension)+cluster_method+".xlsx")
#feature_evaluation_df.to_parquet("Cohort/Models/Feature_evaluation_"+dim_red_method+str(dimension)+cluster_method+".parquet")

In [ ]:
plotTopFeatures(df,df_path,merge_w_supervised,dim_red_method, evaluation_results,n_cluster , 10)

## Further Pipeleine Configurations

In [ ]:
df_path='Cohort/Feature_Extraction/ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab.parquet'
df_name='ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab'
age_filter=True
drop_gender=True
drop_age=True
tune_umap=True 
arr_umap_distance=[0.5]
arr_umap_neighbors=[15,50,100]
check_pca=False
plotting=False
num_scaler_name="MinMaxScaler"
cat_scaler_name='BinaryEncoder'
a_f_decoder=''
a_f_encoder=''
batchsize=''
epochs=''
optimizer=''
loss_function=''
ellbow_method=False
#n_cluster=3
anova=False
t_test=False
chi=False
top_features=40
merge_w_supervised=False
merge_w_inpatient=False
dim_red_method='UMAP'
arr_cluster_method=['kmeans','hierarchical']
arr_dimension=[50,60,70,80,90]
arr_n_cluster=[3,4,5]

In [ ]:
#run experiments for AE in a loop: 
df_path='Cohort/Feature_Extraction/ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab.parquet'
df_name='ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab'
age_filter=True
drop_gender=True
drop_age=True
tune_umap=True
umap_distance=0.1
umap_neighbors=50
check_pca=False
plotting=False
num_scaler_name="MinMaxScaler"
cat_scaler_name='BinaryEncoder'
a_f_decoder='tanh'
a_f_encoder='tanh'
batchsize=1000
epochs=100
optimizer='adam'
loss_function='mse'
ellbow_method=False
#n_cluster=3
anova=False
t_test=False
chi=False
top_features=40
merge_w_supervised=False
merge_w_inpatient=False
arr_dim_red_method=['AE']
arr_cluster_method=['kmeans','hierarchical']
arr_dimension=[50,60,70,80,90]
arr_n_cluster=[3,4,5]
x=0
for r in arr_dim_red_method: 
    dim_red_method=r
    for c in arr_cluster_method :
        cluster_method=c
        for d in arr_dimension:
            dimension=d
            for n in arr_n_cluster:
                n_cluster=n
                x=x+1
                df,evaluation_results=cluster_pipeline(df_path,df_name,age_filter,drop_gender,drop_age,tune_umap,umap_distance,umap_neighbors, check_pca,plotting,num_scaler_name,cat_scaler_name, dim_red_method,dimension,a_f_decoder,a_f_encoder,batchsize,epochs,optimizer,loss_function,cluster_method,ellbow_method,n_cluster,anova,chi,top_features,merge_w_supervised,merge_w_inpatient)
                print(x)
                

In [ ]:
#run experiments in a loop for UMAP: 
df_path='Cohort/Feature_Extraction/ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab.parquet'
df_name='ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab'
age_filter=True
drop_gender=True
drop_age=True
tune_umap=True 
arr_umap_distance=[0.5]
arr_umap_neighbors=[15,50,100]
check_pca=False
plotting=False
num_scaler_name="MinMaxScaler"
cat_scaler_name='BinaryEncoder'
a_f_decoder=''
a_f_encoder=''
batchsize=''
epochs=''
optimizer=''
loss_function=''
ellbow_method=False
#n_cluster=3
anova=False
t_test=False
chi=False
top_features=40
merge_w_supervised=False
merge_w_inpatient=False
dim_red_method='UMAP'
arr_cluster_method=['kmeans','hierarchical']
arr_dimension=[50,60,70,80,90]
arr_n_cluster=[3,4,5]
x=0
for dist in arr_umap_distance: 
    umap_distance=dist
    for neighbors in arr_umap_neighbors: 
        umap_neighbors=neighbors
        for c in arr_cluster_method :
            cluster_method=c
            for d in arr_dimension:
                dimension=d
                for n in arr_n_cluster:
                    n_cluster=n
                    x=x+1
                    df,evaluation_results=cluster_pipeline(df_path,df_name,age_filter,drop_gender,drop_age,tune_umap,umap_distance,umap_neighbors,check_pca,plotting,num_scaler_name,cat_scaler_name,dim_red_method,dimension,a_f_decoder,a_f_encoder,batchsize,epochs,optimizer,loss_function,cluster_method,ellbow_method,n_cluster,anova,chi,top_features,merge_w_supervised,merge_w_inpatient)
                    print(x)


In [ ]:
#run experiments in a loop: 
df_path='Cohort/Feature_Extraction/ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab.parquet'
df_name='ALL_HF_cohort_unsupervised_only_after_onset_HF_ALL_all_any_all_mean_medium_cleaned_wLab'
age_filter=True
drop_gender=True
drop_age=True
tune_umap=True
umap_distance=0.1
umap_neighbors=50
check_pca=False
plotting=False
num_scaler_name="MinMaxScaler"
cat_scaler_name='BinaryEncoder'
a_f_decoder=''
a_f_encoder=''
batchsize=''
epochs=''
optimizer=''
loss_function=''
ellbow_method=False
n_cluster=3
anova=False
t_test=False
chi=False
top_features=40
merge_w_supervised=False
merge_w_inpatient=False
arr_dim_red_method=['PCA','ICA','SVD','UMAP']
arr_cluster_method=['kmeans','hierarchical']
arr_dimension=[50,60,70,80,90]
arr_n_cluster=[3,4,5]
x=0
for r in arr_dim_red_method: 
    dim_red_method=r
    for c in arr_cluster_method :
        cluster_method=c
        for d in arr_dimension:
            dimension=d
            for n in arr_n_cluster:
                n_cluster=n
                x=x+1
                df,evaluation_results=cluster_pipeline(df_path,df_name,age_filter,drop_gender,drop_age,umap_distance,umap_neighbors,check_pca,plotting,num_scaler_name,cat_scaler_name,dim_red_method,dimension,a_f_decoder,a_f_encoder,batchsize,epochs,optimizer,loss_function,cluster_method,ellbow_method,n_cluster,anova,chi,top_features,merge_w_supervised,merge_w_inpatient)
                print(x)
                

## Plotting of the result overvie table 

In [ ]:
import seaborn as sns
evaluation_metrics=pq.read_table('Cohort/Models/Metrics_Results.parquet').to_pandas()
cm = sns.light_palette("green", as_cmap=True)
evaluation_metrics=evaluation_metrics.sort_values([ "calinski_harabasz","silhouette_Coefficient", "davies_bouldin"], ascending = (False, False,True))
s = evaluation_metrics.style.background_gradient(cmap=cm)
s
#s.to_excel("Cohort/Models/10_26_Evaluation.xlsx")

## Creating new Result Array 

In [ ]:
col=['Experiment Name','Dataset','Numerical Scaler','Categorical Scaler','Dimension Reduction Method','Number of Dimension','Activation Function Decoder','Activation Function Encoder','Number of Layer','batchsize','epochs','optimizer','loss function','validation loss','Cluster Method','Number of Cluster','silhouette_Coefficient' , 'calinski_harabasz' , 'davies_bouldin','silhouette_Coefficient_original_Cohort' , 'calinski_harabasz_original_Cohort' , 'davies_bouldin_original_Cohort']
result=pd.DataFrame(columns=col)
result.to_parquet('Cohort/Models/Metrics_Results.parquet')

In [ ]:
evaluation_results=pq.read_table('Cohort/Models/Metrics_Results.parquet').to_pandas()


In [ ]:
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True)
evaluation_results=evaluation_results.sort_values(["silhouette_Coefficient", "calinski_harabasz", "davies_bouldin"], ascending = (False, False,True))
s = evaluation_results.style.background_gradient(cmap=cm)
s